In [1]:
import pandas as pd
import numpy as np
import faiss
from datasets import load_dataset
from faiss import read_index
from tqdm import tqdm

In [5]:
database_index = read_index("../data/faiss_indexes/distiluse-base-multilingual-cased-v2.index")
database_df = pd.read_pickle("../data/database.pkl")
test_df = load_dataset("dkoterwa/oasst2_filtered", split="train").to_pandas()
test_embeddings = np.load("../data/test_embeddings/oasst2_filtered_distiluse-base-multilingual-cased-v2.npy", allow_pickle=True)

Found cached dataset parquet (/home/zeus/.cache/huggingface/datasets/dkoterwa___parquet/dkoterwa--oasst2_filtered-2e17fd8877b64a5f/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [3]:
def find_neighbors_in_index(candidate: np.ndarray, index: faiss.IndexFlat, k: int=1):
    scores, indices = index.search(candidate, k)
    return scores, indices

In [4]:
nearest_neighbors_texts = []
nearest_neighbors_scores = []
for candidate in tqdm(test_embeddings[:100], desc="Retrieving nearest neighbors from index for test embeddings"):
    scores, indices = find_neighbors_in_index(np.expand_dims(candidate, axis=0), database_index, k=1)
    nearest_neighbor_text = database_df.iloc[indices.item()]["text"]
    nearest_neighbors_texts.append(nearest_neighbor_text)
    nearest_neighbors_scores.append(scores.item())
temp_df = test_df.iloc[:100].copy()
temp_df["nearest_neighbor_text"] = nearest_neighbors_texts
temp_df["nearest_neighbor_score"] = nearest_neighbors_scores

Retrieving nearest neighbors from index for test embeddings: 100%|██████████| 100/100 [01:14<00:00,  1.34it/s]
